# Fabric Analytics Roadshow - Deployment Notebook

This notebook deploys all lab assets into your Fabric workspace.

## Prerequisites
- Access to a Microsoft Fabric workspace
- Contributor or Admin permissions in the workspace

## What This Notebook Does
1. Clones Analytics Roadshow GIT Repo
1. Uses fabric-cicd python library to deploy the project to the target workspace

In [1]:
%pip install C:\Users\milescole\source\fabric-analytics-roadshow-lab\setup\fabric_cicd-0.1.33-py3-none-any.whl --force-reinstall

Processing c:\users\milescole\source\fabric-analytics-roadshow-lab\setup\fabric_cicd-0.1.33-py3-none-any.whl
  Using cached azure_identity-1.25.1-py3-none-any.whl.metadata (88 kB)
  Using cached dpath-2.2.0-py3-none-any.whl.metadata (15 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached jsonpath_ng-1.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached azure_core-1.37.0-py3-none-any.whl.metadata (47 kB)
  Using cached cryptography-46.0.3-cp311-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached msal-1.34.0-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.3.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached cffi-2.0.0-cp312-cp312-win_amd64.whl.metadata (2.6 k

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


In [ ]:
from fabric_cicd import FabricWorkspace, publish_all_items, unpublish_all_orphan_items, append_feature_flag

append_feature_flag("enable_lakehouse_unpublish")

# Sample values for FabricWorkspace parameters
workspace_id = "a525eeb6-fb15-43aa-9b28-ce6cab29eb24"
environment = "DEPLOY"
repository_directory = r"C:\Users\milescole\source\fabric-analytics-roadshow-lab\workspace"
item_type_in_scope = ["Notebook", "Lakehouse", "SparkJobDefinition"]  # , "Environment"

# Initialize the FabricWorkspace object with the required parameters
target_workspace = FabricWorkspace(
    workspace_id=workspace_id,
    environment=environment,
    repository_directory=repository_directory,
    item_type_in_scope=item_type_in_scope,
    enable_lakehouse_unpublish=True
)

# Publish all items defined in item_type_in_scope
publish_all_items(target_workspace)

# Unpublish all items defined in item_type_in_scope not found in repository
#unpublish_all_orphan_items(target_workspace)

[info]   16:24:43 - Executing as User 'milescole@microsoft.com'
[warn]   16:24:43 - Found the reserved environment key '_ALL_'
[warn]   16:24:43 - The replace value: '$items.Lakehouse.SalesAndLogisticsLH.$id' will be applied for any target environment
[warn]   16:24:43 - Found the reserved environment key '_ALL_'
[warn]   16:24:43 - The replace value: '$workspace.$id' will be applied for any target environment
[warn]   16:24:43 - Found the reserved environment key '_ALL_'
[warn]   16:24:43 - The replace value: '$items.Lakehouse.SalesAndLogisticsLH.$id' will be applied for any target environment
[warn]   16:24:43 - Found the reserved environment key '_ALL_'
[warn]   16:24:43 - The replace value: '$items.Lakehouse.SalesAndLogisticsLH.$id' will be applied for any target environment
[warn]   16:24:43 - Found the reserved environment key '_ALL_'
[warn]   16:24:43 - The replace value: '$items.Environment.SparkEnv.$id' will be applied for any target environment
[warn]   16:24:43 - The provide


####################################################################################################
########## Validating Parameter File ###############################################################
####################################################################################################



[info]   16:24:45 - Publishing Workspace Folders
         16:24:45 - Published



####################################################################################################
########## Publishing Workspace Folders ############################################################
####################################################################################################



[info]   16:24:47 - Publishing Lakehouse 'SalesAndLogisticsLH'


...

[info]   16:25:00 - Publishing SparkJobDefinition 'StreamBronzeAndSilver'
         16:25:03 - Operation in progress. Checking again in 1 second (Attempt 1)...
         16:25:04 - Operation in progress. Checking again in 2 seconds (Attempt 2)...
         16:25:07 - Published



####################################################################################################
########## Publishing Notebooks ####################################################################
####################################################################################################



[info]   16:25:13 - Publishing Notebook '1_ExploreData'
         16:25:15 - Published
[info]   16:25:15 - Checking Environment Publish State for ['SparkEnv']



####################################################################################################
########## Checking Environment Publish State ######################################################
####################################################################################################



         16:25:16 - Operation in progress. Checking again in 10 seconds (Attempt 1)...
         16:25:27 - Operation in progress. Checking again in 20 seconds (Attempt 2)...
         16:25:47 - Operation in progress. Checking again in 40 seconds (Attempt 3)...
         16:26:28 - Operation in progress. Checking again in 80 seconds (Attempt 4)...
         16:27:49 - Operation in progress. Checking again in 120 seconds (Attempt 5)...
